# Lab 07

In [15]:
import gzip
import tensorflow as tf
import numpy as np
import io
from PIL import Image

In [14]:

def extract_images_from_idx(file_path):
    with gzip.open(file_path, 'rb') as f:
        magic_number = int.from_bytes(f.read(4), byteorder='big')
        num_images = int.from_bytes(f.read(4), byteorder='big')
        rows = int.from_bytes(f.read(4), byteorder='big')
        cols = int.from_bytes(f.read(4), byteorder='big')
        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8).reshape(num_images, rows, cols)
    return images

def extract_labels_from_idx(file_path):
    with gzip.open(file_path, 'rb') as f:
        magic_number = int.from_bytes(f.read(4), byteorder='big')
        num_labels = int.from_bytes(f.read(4), byteorder='big')
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
    return labels

def preprocess_images(images):
    images = np.array(images, dtype=np.float32) / 255.0
    return images

def create_tf_dataset(images, labels, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.shuffle(buffer_size=len(images)).batch(batch_size)
    return dataset

In [17]:
# paths to gzip files
train_image_file_path = 'train-images-idx3-ubyte.gz'
train_label_file_path = 'train-labels-idx1-ubyte.gz'
test_image_file_path = 't10k-images-idx3-ubyte.gz'
test_label_file_path = 't10k-labels-idx1-ubyte.gz'

# Extracting images and labels
train_images = extract_images_from_idx(train_image_file_path)
train_labels = extract_labels_from_idx(train_label_file_path)
test_images = extract_images_from_idx(test_image_file_path)
test_labels = extract_labels_from_idx(test_label_file_path)

# Preprocessing images
train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)

# Creating TensorFlow datasets
batch_size = 32
train_dataset = create_tf_dataset(train_images, train_labels, batch_size)
test_dataset = create_tf_dataset(test_images, test_labels, batch_size)

# Defining TensorFlow model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training the model
model.fit(train_dataset, epochs=24)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')


Epoch 1/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7560 - loss: 0.6964
Epoch 2/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8531 - loss: 0.4071
Epoch 3/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8655 - loss: 0.3687
Epoch 4/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8750 - loss: 0.3450
Epoch 5/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8791 - loss: 0.3314
Epoch 6/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8855 - loss: 0.3145
Epoch 7/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8877 - loss: 0.3034
Epoch 8/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8908 - loss: 0.2984
Epoch 9/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8940 - loss: 0.2890
Epoch 10/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8942 - loss: 0.2843
Epoch 11/24
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8955 - loss: 0.2747
Epoch 12/24
1875/1875 ━━━━━━━━